In [0]:
#!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from pathlib import Path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import seaborn as sns

from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
import pandas as pd
from matplotlib.image import imread
import glob
from numpy import savez_compressed
from sklearn.metrics import fbeta_score
from keras.optimizers import SGD
from keras import backend
import sys

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import h5py
from numpy import load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from PIL import Image
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
import os
import shutil


# Functions


In [0]:
### FUNCTIONS ###

def create_labels_df(data_path):
  ''' This funcitons grabs all files from a particular folder - extracts elements from the filenames and crates a df with filenames in one column and custom tags in another column '''
  # Extract all filenames in folder, get the filenames of image dataset without file extensions
  # Get the text before the '.' in a directory if it is indeed a file
  image_labels = [f.split('.')[0] for f in listdir(data_path) if isfile(join(data_path, f))]

  # Create tags for each image class (remove filename text after '__' to exclude augmentation and jpg)
  image_tags = [f.split('__')[0] for f in listdir(data_path) if isfile(join(data_path, f))]

  # Create a dataframe from image_labels and image_tags
  labels_df = pd.DataFrame()
  labels_df['Filenames'] = image_labels
  labels_df['Tags'] = image_tags
  print(labels_df.head())
  return labels_df

####################################

def create_tag_mapping(labels_df):
  ''' This function takes the tags from a column in a df - maps integers to each tag and vice verca '''
  # Create labels for tags
  labels = set()
  for i in range(len(labels_df)):
    # convert spaced separated tags into an array of tags
    tags = labels_df['Tags'][i]#.split(' ') # split tag if necessary
    # add tags to the set of known labels
    labels.update(tags)
  
  # Convert labels set into a list
  labels = list(labels)
  # Sort alphabetically
  labels.sort()

  # Create dict that maps labels to integers and reverse
  labels_map = {labels[i] : i for i in range(len(labels))}
  inv_labels_map = {i : labels[i] for i in range(len(labels))}
  return labels_map, inv_labels_map

####################################

def create_file_mapping(labels_df, filenames_col, tags_col):
  mapping = dict()
  for i in range(len(labels_df)):
    name, tags = labels_df[filenames_col][i], labels_df[tags_col][i]
    mapping[name] = tags#.split(' ')
  return mapping

####################################

def one_hot_encode_single(tags, tag_to_int_map):
  # create empty vector.
	encoding = np.zeros(len(tag_to_int_map), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[tag_to_int_map[tag]] = 1
	return encoding

####################################

def oneHotEncode(path_to_files):
  ''' Create one hot encodings for an ENTIRE LIST of tags'''
  # get the filenames of image dataset without file extensions
  # use set() to get only unique filenames
  unique_image_labels = set([f.split('__')[0] for f in listdir(path_to_files) if isfile(join(path_to_files, f))])
  list_image_labels = [f.split('__')[0] for f in listdir(path_to_files) if isfile(join(path_to_files, f))]

  ### One-hot-encode label names in dataset ###
  # The number of image categories - unique_image_labels
  n_categories = len(unique_image_labels)

  # The unique values of categories in the data
  categories = np.array(unique_image_labels)

  # Initialize ohe_labels as all zeros
  ohe_labels = np.zeros((len(image_labels), n_categories))

  # Loop over the labels
  for ii in range(len(image_labels)):
      # Find the location of this label in the categories variable
      jj = np.where(categories == list_image_labels[ii])
      # Set the corresponding zero to one
      ohe_labels[ii, jj] = 1
  return ohe_labels

###################################

def loadImages(path_to_images, image_filenames, IMG_HEIGHT, IMG_WIDTH):
  '''Load image files from a directory into a variable as numpy arrays'''
  # Create object to store images after importing and processing
  loaded_images = []

  #Load in and process images
  for i in tqdm(range(len((image_filenames)))):
    path = path_to_images + image_filenames[i] + '.jpg'
    img = image.load_img(path, target_size=(IMG_WIDTH, IMG_HEIGHT, 3))
    img = image.img_to_array(img)
    img = img/255.0
    loaded_images.append(img)

  loaded_images = np.array(loaded_images)
  return loaded_images

###################################

def loadDataset_training(path_to_images, IMG_HEIGHT, IMG_WIDTH):
  ''' Searach directory for files, extract filenames and tags into a df, Load each image, convert to ndarray, one hot encode filenames, save image into variable and corresponding encoded filename into another variable'''
  # Extract all filenames and tags into a df
  labels_df = create_labels_df(path_to_images)
  
  # Create objects to store each processed image and one hot encoded tag
  processed_images = list()

  for image_file in listdir(path_to_images):
    # Define image path and filename
    path = path_to_images + image_file
    
    # Load an image and convert to ndarray
    loaded_image = image.img_to_array(image.load_img(path, target_size=(IMG_WIDTH, IMG_HEIGHT, 3)))

    # Get corresponding tag from dataframe containing filenames and tags info
    tags = labels_df['Tags']

    # One hot encode corresponding tag
    #target = oneHotEncode(path_to_images)

    # Save image and ohe'd tag into separate variables as arrays
    processed_images.append(loaded_image)
  
  # One hot encode the filenames
  encoded_filenames = oneHotEncode(path_to_images)
  # Save to 
  X = np.asarray(processed_images, dtype='uint8')
  y = encoded_filenames
  # save both arrays to one file in compressed format
  savez_compressed('bike_parts.npz', X, y)

  # Separate into train and test datasets
  trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
  print(trainX.shape, trainY.shape, testX.shape, testY.shape)
  
  return trainX, testX, trainY, testY

#########################################################
def loadDataset(path_to_images, IMG_HEIGHT, IMG_WIDTH):
  ''' Searach directory for files, extract filenames and tags into a df, Load each image, convert to ndarray, one hot encode filenames, save image into variable and corresponding encoded filename into another variable'''
  # Extract all filenames and tags into a df
  labels_df = create_labels_df(path_to_images)
  
  # Create objects to store each processed image and one hot encoded tag
  processed_images = list()

  for image_file in listdir(path_to_images):
    # Define image path and filename
    path = path_to_images + image_file
    
    # Load an image and convert to ndarray
    loaded_image = image.load_img(path, target_size=(IMG_WIDTH, IMG_HEIGHT, 3))
    loaded_image = image.img_to_array(loaded_image)
    

    # Get corresponding tag from dataframe containing filenames and tags info
    tags = labels_df['Tags']

    # One hot encode corresponding tag
    #target = oneHotEncode(path_to_images)

    # Save image and ohe'd tag into separate variables as arrays
    processed_images.append(loaded_image)
  
  # One hot encode the filenames
  encoded_filenames = oneHotEncode(path_to_images)
  # Save to 
  X = np.asarray(processed_images, dtype='uint8')
  y = encoded_filenames
  # save both arrays to one file in compressed format
  savez_compressed('bike_parts.npz', X, y)
#########################################################

def decode_predictions(path_to_files):
  '''Function to map original filenames to ohe_labels, then decode predictions into original image filenames'''
  # Get image labels from files in directory
  image_labels = [f.split('.')[0] for f in listdir(path_to_files) if isfile(join(path_to_files, f))]
  # Generate ohe_labels from filenames in directory
  ohe_labels = oneHotEncode(training_data_path)
  # Map image labels to ohe array in dictionary
  mapping = dict()
  for i in range(len(image_labels)):
    label, ohe = image_labels[i], ohe_labels[i]
    mapping[label] = ohe

  # Get the indices of top 3 predictions
  top3 = np.argsort(predictions[0])[:-4:-1] 
  
  # Get ohe_labels based on indices in top3
  top3_ohe_labels = []

  for i in range(3):
    top3_ohe_labels.append(ohe_labels[top3[i]])
  # Turn ohe_labels list into an array
  top3_array = np.array(top3_ohe_labels)
  # Get image label corresponding to ohe_label in top3 predictions
  pred_labels = []
  for label, value in mapping.items():
    for item in top3_array:
      if np.array_equal(value, item):
        pred_labels.append(label)
  return pred_labels



In [0]:
# Mount the google drive folder to colab notebook
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Image_Recommender/' # folder in gdrive

# Define location datasets
training_data_folder = 'gdrive/My Drive/Image_Recommender/Original/full_dataset/'
augmented_data_path = 'gdrive/My Drive/Image_Recommender/Augmented_large/'
test_folder_path = 'gdrive/My Drive/Image_Recommender/Test_images/'
training_checkpoints_path = 'gdrive/My Drive/Image_Recommender/training_checkpoints/'
augmented_full_dataset_path = 'gdrive/My Drive/Image_Recommender/Augmented_full_dataset/'
augmented2 = 'gdrive/My Drive/Image_Recommender/Augmented2/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
print(tf.__version__)
sns.set()

2.0.0-beta1


# Sort files in drive


In [0]:
def sort_files(parent_dir, daughter_dir):
  """ 
  This function creates a new daughter directory within the parent directory, then sorts files with the same filenames inside the parent directory into filename-specific folders inside the daughter directory
  """
  # Define the dir paths and create a new sub-dir if it does not already exist
  if not os.path.exists(daughter_dir):
    os.makedirs(daughter_dir)
  
  # Iterate through all files inside parent folder
  for imagefile in os.listdir(parent_dir):
    if os.path.isfile(parent_dir + imagefile) == True:
      if fnmatch.fnmatch(imagefile, '*.jpg'):
        
        try:
          # Grab filenames, paths 
          filename = imagefile.split('.jpg')[0]
          if not os.path.exists(daughter_dir + filename):   
            new_file_dir = daughter_dir + filename 
        except Exception as err:
          print("Error when grabbing filenames {}".format(err))   
        
        # Check if directory exists or not 
        if not os.path.exists(new_file_dir):
          try:
            os.makedirs((new_file_dir))
          except Exception as err:
            print('Error occurred during new folder creation err :{}'.format(err)) 
        
        # If directory exists, move the file into its respective folder
        if os.path.exists(new_file_dir):
            try:
              old_file_dir = parent_dir + imagefile 
              shutil.move(old_file_dir, new_file_dir)
              print('Moved :{}'.format(imagefile))
            except Exception as err:
              print('Error occurred during moving :{}'.format(err))
  
  return print('All files sorted')


In [0]:
parent_dir = 'gdrive/My Drive/Image_Recommender/Original/full_dataset/'
daughter_dir = 'gdrive/My Drive/Image_Recommender/' + 'Sorted_images/'
sort_files(parent_dir, daughter_dir)

All files sorted


In [0]:
print(len(os.listdir(daughter_dir)))

8141


# Augment images

In [0]:
def augment_images(training_data_path, augmented_data_path):
  #### WORKING CODE FOR IMAGE AUGMENTATIONS ####
  
  for imagefile in listdir(training_data_path):
    
    try:
      # Get the image filename
      image_label = imagefile.split('.')[0]
    except:
      print(imagefile, ' at extracting image label')

    try:
      # Load an image with imageio (expand to many images later)
      loaded_image = imageio.imread(training_data_path + imagefile)
    except:
      print(imagefile, ' at loading image')
    
    try:
      # Save the original image in the augmented folder
      im = Image.fromarray(loaded_image)
      im.save(augmented_data_path + image_label, 'JPEG')
    except:
      print(imagefile, ' save original image')

    try:
      # Apply motionblur and save into a new object then save to disk
      img_motionblur = iaa.MotionBlur(k=15, angle=45)(images=loaded_image)
      im = Image.fromarray(img_motionblur)
      im.save(augmented_data_path + image_label + '__motionblur', 'JPEG')
    except:
      print(imagefile, ' at motionblur')

    try:
      # Apply flip and save into a new object then save to disk
      img_flipped = iaa.Flipud(1.0)(images=loaded_image)
      im = Image.fromarray(img_flipped)
      im.save(augmented_data_path + image_label + '__flipped', 'JPEG')
    except:
      print(imagefile, ' at flipped')

    try:
      # Apply sharpen (detect outlines) and save into a new object then save to disk
      img_sharpen = iaa.Sharpen(alpha=1, lightness=0)(images=loaded_image)
      im = Image.fromarray(img_sharpen)
      im.save(augmented_data_path + image_label + '__sharpen', 'JPEG')
    except:
      print(imagefile, ' at sharpen')

    try:
      # Apply a flipud (horizontal) + sharpen to each image then save to disk
      img_flip_sharpen = iaa.Sequential([
          iaa.Flipud(1.0),
          iaa.Sharpen(alpha=1, lightness=0)])(images=loaded_image)
      im = Image.fromarray(img_flip_sharpen)
      im.save(augmented_data_path + image_label + '__img_flip_sharpen', 'JPEG')
    except:
      print(imagefile, ' at flipud + sharpen')

    try:
      # Apply a motionblur then sharpen to each image then save to disk
      img_motionblur_sharpen = iaa.Sequential([
                                  iaa.MotionBlur(k=15, angle=45),
                                  iaa.Sharpen(alpha=1, lightness=0)])(images=loaded_image)
      im = Image.fromarray(img_motionblur_sharpen)
      im.save(augmented_data_path + image_label + '__motionblur_sharpen', 'JPEG')
    except:
      print(imagefile, ' at motionblur + sharpen')

# RUN IT!
#augment_images(training_data_folder, augmented2)


zadelpen-hevelmoer-m8.png  save original image
zadelpen-hevelmoer-m8.png  at motionblur
zadelpen-hevelmoer-m8.png  at flipped
zadelpen-hevelmoer-m8.png  at sharpen
zadelpen-hevelmoer-m8.png  at flipud + sharpen
zadelpen-hevelmoer-m8.png  at motionblur + sharpen
sks-spanningmeter-airchecker-3-5-10-bar.png  save original image
sks-spanningmeter-airchecker-3-5-10-bar.png  at motionblur
sks-spanningmeter-airchecker-3-5-10-bar.png  at flipped
sks-spanningmeter-airchecker-3-5-10-bar.png  at sharpen
sks-spanningmeter-airchecker-3-5-10-bar.png  at flipud + sharpen
sks-spanningmeter-airchecker-3-5-10-bar.png  at motionblur + sharpen
spanninga-koplamp-trigon-10-luxe-vork-bevestiging.png  save original image
spanninga-koplamp-trigon-10-luxe-vork-bevestiging.png  at motionblur
spanninga-koplamp-trigon-10-luxe-vork-bevestiging.png  at flipped
spanninga-koplamp-trigon-10-luxe-vork-bevestiging.png  at sharpen
spanninga-koplamp-trigon-10-luxe-vork-bevestiging.png  at flipud + sharpen
spanninga-koplamp

In [0]:
##### LATEST PIPELINE #####

def create_labels_df(data_path):
  ''' This funcitons grabs all files from a particular folder - extracts elements from the filenames and crates a df with filenames in one column and custom tags in another column '''
  # Extract all filenames in folder, get the filenames of image dataset without file extensions
  # Get the text before the '.' in a directory if it is indeed a file
  image_labels = [f.split('.')[0] for f in listdir(data_path) if isfile(join(data_path, f))]

  # Create tags for each image class (remove filename text after '__' to exclude augmentation and jpg)
  image_tags = [f.split('__')[0] for f in listdir(data_path) if isfile(join(data_path, f))]

  # Create a dataframe from image_labels and image_tags
  labels_df = pd.DataFrame()
  labels_df['Filenames'] = image_labels
  labels_df['Tags'] = image_tags
  print(labels_df.head())
  return labels_df

def create_tag_mapping(labels_df):
  ''' This function takes the tags from a column in a df - maps integers to each tag and vice verca '''
  # Create labels for tags
  labels = set()
  for i in range(len(labels_df)):
    # convert spaced separated tags into an array of tags
    tags = labels_df['Tags'][i].split(' ') # use split to ensure the whole tag, not jsut single characters, are plaved into set() object
    # add tags to the set of known labels
    labels.update(tags)
  
  # Convert labels set into a list
  labels = list(labels)
  # Sort alphabetically
  labels.sort()

  # Create dict that maps labels to integers and reverse
  labels_map = {labels[i] : i for i in range(len(labels))}
  inv_labels_map = {i : labels[i] for i in range(len(labels))}
  return labels_map, inv_labels_map

####################################

def create_file_mapping(labels_df, filenames_col, tags_col):
  mapping = dict()
  for i in range(len(labels_df)):
    name, tags = labels_df['Filenames'][i], labels_df['Tags'][i]
    mapping[name] = tags#.split(' ')
  return mapping

# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
	# create empty vector
	encoding = np.zeros(len(mapping), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[mapping[tags]] = 1
	return encoding

# load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
	photos, targets = list(), list()
	# enumerate files in the directory
	for filename in listdir(folder):
		# load image
		photo = load_img(path + filename, target_size=(128,128))
		# convert to numpy array
		photo = img_to_array(photo, dtype='uint8')
		# get tags
		tags = file_mapping[filename]
		# one hot encode tags
		target = one_hot_encode(tags, tag_mapping)
		# store
		photos.append(photo)
		targets.append(target)
	X = np.array(photos, dtype='uint8')
	y = np.array(targets, dtype='uint8')
	return X, y

labels_df = create_labels_df(augmented_full_dataset_path)
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(labels_df)
# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(labels_df, 'Filenames', 'Tags')
# load the jpeg images
folder = augmented_data_path
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
# save both arrays to one file in compressed format
savez_compressed('bike_parts.npz', X, y)

                                           Filenames                                             Tags
0  bafang-schijfring-met-nok-voor-front-hub-motor...   bafang-schijfring-met-nok-voor-front-hub-motor
1  bafang-schijfring-met-nok-voor-front-hub-motor...   bafang-schijfring-met-nok-voor-front-hub-motor
2  bafang-schijfring-met-nok-voor-front-hub-motor...   bafang-schijfring-met-nok-voor-front-hub-motor
3  bafang-lader-acculader-43-volt-2a-chg-c01-2a-e...  bafang-lader-acculader-43-volt-2a-chg-c01-2a-en
4  bafang-lader-acculader-43-volt-2a-chg-c01-2a-e...  bafang-lader-acculader-43-volt-2a-chg-c01-2a-en


KeyError: ignored

# Model using CGG from ImageNet with centered image pixels

# Final Model

In [0]:
# load train and test dataset
def load_dataset():
	# load dataset
	data = load('bike_parts.npz')
	X, y = data['arr_0'], data['arr_1']
	return X, y
 
# # calculate fbeta score for multi-class/label classification
# def fbeta(y_true, y_pred, beta=2):
# 	# clip predictions
# 	y_pred = backend.clip(y_pred, 0, 1)
# 	# calculate elements
# 	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
# 	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
# 	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
# 	# calculate precision
# 	p = tp / (tp + fp + backend.epsilon())
# 	# calculate recall
# 	r = tp / (tp + fn + backend.epsilon())
# 	# calculate fbeta, averaged across each class
# 	bb = beta ** 2
# 	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
# 	return fbeta_score
 
# define cnn model
def define_model(in_shape=(128, 128, 3), out_shape=388):
	# load model
	model = VGG16(include_top=False, input_shape=in_shape)
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# allow last vgg block to be trainable
	# model.get_layer('block5_conv1').trainable = True
	# model.get_layer('block5_conv2').trainable = True
	# model.get_layer('block5_conv3').trainable = True
	# model.get_layer('block5_pool').trainable = True
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(out_shape, activation='softmax')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = 'adam' #SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy')
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	plt.show()
  # plot accuracy
	plt.subplot(212)
	plt.title('Fbeta')
	plt.plot(history.history['fbeta'], color='blue', label='train')
	plt.plot(history.history['val_fbeta'], color='orange', label='test')
	plt.show()
  # save plot to file
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.close()
 
def run_test_harness():
	
	# load and process images and filenames then save into .npz file
	#loadDataset(training_data_path, 128, 128)
	# load dataset from saved npz file
	X, y = load_dataset()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow(X, y, batch_size=128)
	# define model
	model = define_model()
	
	# Save training progress
	checkpoint_path = training_checkpoints_path
	checkpoint_dir = os.path.dirname(checkpoint_path)

	# Create a callback that saves the model's weights
	cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
																									save_weights_only=True,
																									verbose=1)
	# fit model
	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=50, verbose=1, callbacks=[cp_callback])
	# save model
	model.save('final_model.h5')
  

# entry point, run the test harness
run_test_harness()

FileNotFoundError: ignored

# Predict on new image

In [0]:
	# convert a prediction to tags
def top3_predictions(inv_mapping, predictions):
	# Get indices of top 3 results from predictions array
	top3 = np.argsort(predictions)[:-4:-1] 
	print('Indices of top 3 predictions ', top3)
	# collect all predicted tags based on indices extracted in top3
	tags = [inv_mapping[i] for i in top3]
	#print('Predicted Tags ', tags)	
	return tags
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(128, 128))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 128, 128, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 
# load an image and predict the class
def run_example(inv_mapping, chosen_image):
	# load the image
	img = load_image(chosen_image)
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	#print(result[0])
	# map prediction to tags
	tags = top3_predictions(inv_mapping, result[0])
	print('Predicted Tags ', tags)
 
# Set filepath for image to predict
chosen_image = test_folder_path + '3379399395040_01.jpg'

# create a mapping of tags to integers from labels_df containing 'Filenames' and 'Tags'
_, inv_mapping = create_tag_mapping(labels_df)
# entry point, run the example
run_example(inv_mapping, chosen_image)

Indices of top 3 predictions  [286  39 337]
Predicted Tags  ['spanninga-koplamp-kendo-led-aan-uit-15-lux', 'bofix-flensmoer-m6-100-stuks', 'union-derailleur-pad-gh-225']


# Convert top 3 predictions to filename or tag ID

# Image augmentation pipeline setup
